# Spam Detection

## Introduction

Spamming is one of the simplest attacks in email communication. Users often receive annoying spam messages and malicious phishing emails by subscribing to various websites, products, services, directories, newsletters, and other types of electronic communication. In some cases, spam emails are generated by viruses or trojan horses sent en masse.

There are many solutions for spam filtering, such as blacklisting and whitelisting techniques, decision tree-based approaches, email address-based approaches, and machine learning-based methods. Most of them rely mainly on the analysis of the email content's text. As a result, there is an increasing demand for effective anti-spam filters that automatically identify and remove spam messages or warn users about potential spam messages. However, spammers always explore the gaps in existing spam filtering techniques and introduce new designs to spread spam widely, e.g., the tokenization attack sometimes misleads spam filters by adding extra spaces. Therefore, the content of emails needs to be structured. Moreover, despite having the highest accuracy in detecting spam using machine learning, false positives (FP) are a problem due to one-time email threat detection. To address the issues of false positives and changes in various attack designs, keywords and other unwanted information are removed from the text before further analysis. After preprocessing, these texts go through numerous feature extraction methods, such as word2vec, word n-gram, character n-gram, and combinations of n-grams with variable lengths. Various machine learning techniques, such as support vector machine (SVM), decision tree (DT), logistic regression (LR), and multinomial naive Bayes (MNB), are used to classify emails.

In this task, we will focus only on the naive Bayes classifier method presented in the lecture along with the [multinomial version](https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Multinomial_naive_Bayes).

## Naive Bayes Classifiers (NB)

In our experiments, after preprocessing, each message is ultimately represented as a vector $\mathbf{x}=(x_1, \ldots , x_m)$, where $x_1, \ldots , x_m$ are the attribute values $X_1, \ldots , X_m$, and each attribute provides information about a specific token of the message. In the simplest case, all attributes are boolean values: $X_i = 1$ if the message contains the given token; otherwise, $X_i = 0$. Alternatively, their values can be token frequencies (TF), showing how many times the corresponding token appears in the message. Attributes with TF values carry more information than boolean attributes.

According to Bayes' theorem, the probability that a message with vector $\mathbf{x} = (x_1, \ldots, x_m)$ belongs to category $c$ is:

$$
p(c | \mathbf{x}) = \frac{p(c) \cdot p(\mathbf{x} | c)}{p(\mathbf{x})}
$$

Since the denominator does not depend on the category, the NB classifier classifies each message to the category that maximizes $p(c) \cdot p(\mathbf{x} | c)$. In the case of spam filtering, this means classifying a message as spam when:

$$
\frac{p(c_s) \cdot p(\mathbf{x} | c_s)}{p(c_s) \cdot p(\mathbf{x} | c_s) + p(c_h) \cdot p(\mathbf{x} | c_h)} > T
$$

where $T = 0.5$, and $c_h$ and $c_s$ denote the categories ham and spam. By changing $T$, one can choose to have more true negatives (correctly classified ham messages) at the cost of fewer true positives (correctly classified spam messages), or vice versa. The prior probabilities $p(c)$ are usually estimated by dividing the number of training messages in category $c$ by the total number of training messages. The probabilities $p(\mathbf{x} | c)$ are estimated differently in each version of NB - see the lecture.

## Multinomial Naive Bayes Classifier (MNB)

The [multinomial](https://en.wikipedia.org/wiki/Multinomial_distribution) naive Bayes classifier with TF attributes treats each message $d$ as a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) containing each token $t_i$ as many times as it appears in $d$. Therefore, $d$ can be represented as $\mathbf{x} = (x_1, ..., x_m)$, where each $x_i$ is now the number of occurrences of $t_i$ in $d$. Additionally, each message $d$ from category $c$ is viewed as a result of independently choosing $|d|$ tokens from $F=\{t_1,\ldots,t_m\}$ with replacement, with probability $p(t_i | c)$ for each $t_i$. Then $p(\mathbf{x} | c)$ is the multinomial distribution:

$$
p(\mathbf{x} \mid c) = p(|d|) \cdot |d|! \cdot \prod_{i=1}^{d} \frac{p(t_i \mid c)^{x_i}}{x_i !}
$$

where we assume that $|d|$ does not depend on the category $c$. This is an additional simplifying assumption that is more debatable in spam filtering. For example, the probability of receiving a very long spam message seems smaller than the probability of receiving an equally long ham message. The criterion for classifying a message as spam becomes:

$$
\frac{p(c_s) \cdot \prod_{i=1}^{m} p(t_i \mid c_s)^{x_i}}{p(c_s)\cdot\prod_{i=1}^{m} p(t_i \mid c_s)^{x_i} + p(c_h)\cdot\prod_{i=1}^{m} p(t_i \mid c_h)^{x_i}}  > T
$$

where each $p(t_i | c)$ is estimated as:

$$
p(t \mid c) = \frac{\alpha + N_{t,c}}{\alpha \cdot m + N_c}
$$

where $N_{t,c}$ is the number of occurrences of token $t$ in the training messages of category $c$, while $N_c = \sum_{i=1}^{m} N_{t_i,c}$ is the total number of messages in the training set of category $c$. In practice, a parameter $\alpha$ is added, which represents smoothing and solves the problem of zero probability, see [http://www.paulgraham.com/spam.html](http://www.paulgraham.com/spam.html) (e.g., $\alpha=1$).

### Example Multinomial Data

Thus, each message $d$ consists of different tokens $t_i$, and each of these $t_i$ belongs to the dictionary $\mathcal{V}$. If $\mathcal{V}$ contains, for example, $8$ tokens, $t_1,t_2,...,t_8$, and the message is: $t_1 t_2 t_2 t_6 t_3 t_2 t_8$, the representation of this message will be as follows:

| |$t_1$|$t_2$|$t_3$|$t_4$|$t_5$|$t_6$|$t_7$|$t_8$|
|---|---|---|---|---|---|---|---|---|
|$\mathbf{x}$| 1|3 |1 | 0| 0|1 | 0|1 |

After adding a few other random messages, the dataset looks like this:

|$t_1$|$t_2$|$t_3$|$t_4$|$t_5$|$t_6$|$t_7$|$t_8$|$c$|
|---|---|---|---|---|---|---|---|---|
| 1|3 |1 | 0| 0|1 | 0|1 | spam|
| 1|0 |0 | 0| 1|1 | 1|3 | ham|
| 0|0 |0 | 0| 0|2 | 1|2 | spam|

Assuming the classes ($1$-spam, $0$-ham), we have $c = [1,0,1]$. Now, comparing with the equation above:

- $N_{t_i,c}$ is the number of occurrences of feature $t_i$ in each unique class $c$. For example, for $c=1$, $N_{t_1,c}=1, N_{t_6,c}=3$
- $N_c$ is the total number of occurrences of all features in each unique class $c$. For example, for $c=1$, $N_c=12$
- $m=8$ is the total number of features
- $\alpha=1$ is known as the smoothing parameter. It is needed to address the zero probability problem (see [http://www.paulgraham.com/spam.html](http://www.paulgraham.com/spam.html))

## Floating Point Underflow

To avoid the problem of floating point underflow, multiplying a set of small probabilities, i.e., simply the product becomes too small to represent and is replaced by 0. Instead of calculating

$$
P(c) \prod_{i=1}^m P(t_i | c)
$$

which may cause underflow, consider calculating the logarithm of this expression,

$$
\log\left(P(c) \prod_{i=1}^m P(t_i | c)\right)
$$

which equivalently can be written as

$$
\log(P(c))+ \sum_{i=1}^m \log(P(t_i | c))
$$

Then notice that if

$$
\log(P(c_s))+ \sum_{i=1}^m \log(P(t_i | c_s)) > \log(P(c_h))+ \sum_{i=1}^m \log(P(t_i | c_h))
$$

then, since $\log(x) > \log(y)$ if and only if $x > y$, it follows that

$$
P(c_s) \prod_{i=1}^m P(t_i | c_s) > P(c_h) \prod_{i=1}^m P(t_i | c_h)
$$

## Task 1

### Classifier Based on the NB Algorithm

#### Goal:
Build a simple spam classifier based on NB that can detect and filter out unwanted email messages.

#### Description:
1. Collect a dataset containing labels (spam/non-spam) and the content of email messages, e.g., [Enron-Spam](http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/index.html) or [SMS Spam Collection](https://archive.ics.uci.edu/dataset/228/sms+spam+collection) or [E-mail Spam](https://www.kaggle.com/datasets/balaka18/email-spam-classification-dataset-csv) or similar.
2. Prepare the data by tokenizing words and removing unnecessary punctuation marks.
3. Implement NB that can classify messages as spam or non-spam based on the words present.
4. Split the data into a training set and a test set (e.g., 70% for training, 30% for testing).
5. Train the NB classifier on the training data.
6. Test the classifier on the test data and evaluate its effectiveness using metrics: [precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall), [f1-score](https://en.wikipedia.org/wiki/F-score), and [accuracy](https://en.wikipedia.org/wiki/Accuracy_and_precision).
7. Analyze the results and present conclusions.

## Task 2

### Classifier Based on MNB with N-grams

#### Goal:
Build a spam classifier using n-grams in combination with MNB to improve the effectiveness of email message classification.

#### Description:
1. Collect a dataset containing labels (spam/non-spam) and the content of email messages, e.g., [Enron-Spam](http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/index.html) or [SMS Spam Collection](https://archive.ics.uci.edu/dataset/228/sms+spam+collection) or [E-mail Spam](https://www.kaggle.com/datasets/balaka18/email-spam-classification-dataset-csv) or similar.
2. Prepare the data by creating n-grams from the email content, i.e., unigrams, bigrams, trigrams.
3. Implement MNB that can classify messages as spam or non-spam using n-grams as features.
4. Split the data into a training set and a test set (e.g., 70% for training, 30% for testing).
5. Train the MNB classifier on the training data using n-grams as features.
6. Test the classifier on the test data and evaluate its effectiveness using metrics: [precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall), [f1-score](https://en.wikipedia.org/wiki/F-score), and [accuracy](https://en.wikipedia.org/wiki/Accuracy_and_precision).
7. Analyze the results and compare them with the results of the word-based classifier.
8. Present conclusions regarding the effectiveness of the n-gram-based classifier and the impact of different types of n-grams on classification effectiveness.

Imports

In [1]:
import os
import string
import numpy as np

from collections import Counter
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

Models

In [2]:
class NaiveBayesClassifier:
    def __init__(self):
        self.spam_prob = 0
        self.spam_count = 0
        self.ham_count = 0
        self.spam_occurences = Counter()
        self.ham_occurences = Counter()
        self.vocabulary = set()

    def train(self, data, labels):
        self.spam_prob = sum(1 for label in labels if label == 1) / len(labels)
        for i in tqdm(range(len(data)), desc="Training", total=len(labels)):
            tokens = self._tokenize(data[i])
            for token in set(tokens):
                if labels[i]:
                    self.spam_occurences[token] += 1
                else:
                    self.ham_occurences[token] += 1
                self.vocabulary.add(token)
            if labels[i]:
                self.spam_count += 1
            else:
                self.ham_count += 1

    def predict(self, data):
        results = []
        for document in tqdm(data, desc="Predicting", total=len(data)):
            tokens = self._tokenize(document)
            results.append(self._decide_if_spam(tokens))
        return results

    def _tokenize(self, document):
        tokens = word_tokenize(document)
        tokens = [token.lower() for token in tokens]
        table = str.maketrans("", "", string.punctuation)
        tokens = [token.translate(table) for token in tokens]
        tokens = [token for token in tokens if len(token) >= 2]

        return tokens

    def _decide_if_spam(self, document):
        vocab_size = len(self.vocabulary)
        spam_value = np.log(self.spam_prob)
        ham_value = np.log(1 - self.spam_prob)

        for token, count in Counter(document).items():
            token_spam_prob = (1 + self.spam_occurences.get(token, 0)) / (vocab_size + self.spam_count)
            token_ham_prob = (1 + self.ham_occurences.get(token, 0)) / (vocab_size + self.ham_count)

            spam_value += np.log(token_spam_prob) * count
            ham_value += np.log(token_ham_prob) * count
        return spam_value >= ham_value

In [3]:
class MNBClassifier:
    def __init__(self, ngram_size=1):
        self.ngram_size = ngram_size
        self.spam_prob = 0
        self.spam_tokens_count = 0
        self.ham_tokens_count = 0
        self.spam_occurences = Counter()
        self.ham_occurences = Counter()
        self.vocabulary = set()

    def train(self, data, labels):
        self.spam_prob = sum(1 for label in labels if label == 1) / len(labels)
        for i in tqdm(range(len(data)), desc="Training", total=len(labels)):
            tokens = self._tokenize(data[i])
            for token in tokens:
                if labels[i]:
                    self.spam_occurences[token] += 1
                    self.spam_tokens_count += 1
                else:
                    self.ham_occurences[token] += 1
                    self.ham_tokens_count += 1
                self.vocabulary.add(token)

    def predict(self, data):
        results = []
        for document in tqdm(data, desc="Predicting", total=len(data)):
            tokens = self._tokenize(document)
            results.append(self._decide_if_spam(tokens))
        return results

    def _tokenize(self, document):
        tokens = word_tokenize(document)
        tokens = [token.lower() for token in tokens]
        table = str.maketrans("", "", string.punctuation)
        tokens = [token.translate(table) for token in tokens]
        tokens = [token for token in tokens if len(token) >= 2]

        for i in range(2, self.ngram_size + 1):
            n_grams = list(ngrams(tokens, i))
            tokens.extend([" ".join(gram) for gram in n_grams])

        return tokens

    def _decide_if_spam(self, document):
        vocab_size = len(self.vocabulary)
        spam_value = np.log(self.spam_prob)
        ham_value = np.log(1 - self.spam_prob)

        for token, count in Counter(document).items():
            token_spam_prob = (1 + self.spam_occurences.get(token, 0)) / (vocab_size + self.spam_tokens_count)
            token_ham_prob = (1 + self.ham_occurences.get(token, 0)) / (vocab_size + self.ham_tokens_count)

            spam_value += np.log(token_spam_prob) * count
            ham_value += np.log(token_ham_prob) * count
        return spam_value >= ham_value

Metrics

In [4]:
def evaluate_performance(labels, predictions):
    f1_score, precision, recall, accuracy = 0, 0, 0, 0
    tp, tn, fp, fn = 0, 0, 0, 0
    for i in range(len(labels)):
        if labels[i]:
            if predictions[i]:
                tp += 1
            else:
                fn += 1
        else:
            if predictions[i]:
                fp += 1
            else:
                tn += 1
    precision = tp / (tp + fp) if (tp + fp) else 0
    recall = tp / (tp + fn) if (tp + fn) else 0
    accuracy = (tp + tn) / len(labels)
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

    return f1_score, accuracy, recall, precision

Loading data

In [5]:
data = []
labels = []

spam_path = "datasets/enron1/spam"
ham_path = "datasets/enron1/ham"

for file in os.listdir(spam_path):
    labels.append(1)
    with open(os.path.join(spam_path, file), "r", encoding="utf-8", errors="ignore") as file:
        data.append(file.read())
for file in os.listdir(ham_path):
    labels.append(0)
    with open(os.path.join(ham_path, file), "r", encoding="utf-8", errors="ignore") as file:
        data.append(file.read())

train_indices, test_indices = train_test_split(range(len(data)), test_size=0.3, random_state=42)

train_data = [data[i] for i in train_indices]
train_labels = [labels[i] for i in train_indices]
test_data = [data[i] for i in test_indices]
test_labels = [labels[i] for i in test_indices]

Training

In [6]:
nb = NaiveBayesClassifier()
mnb = MNBClassifier()
mnb2 = MNBClassifier(ngram_size=2)
mnb3 = MNBClassifier(ngram_size=3)

nb.train(train_data, train_labels)
mnb.train(train_data, train_labels)
mnb2.train(train_data, train_labels)
mnb3.train(train_data, train_labels)

Training: 100%|██████████| 3620/3620 [00:08<00:00, 406.77it/s]


Evaluation

In [7]:
nb_predictions = nb.predict(test_data)
mnb_predictions = [mnb.predict(test_data), mnb2.predict(test_data), mnb3.predict(test_data)]

Predicting: 100%|██████████| 1552/1552 [00:08<00:00, 181.51it/s]


In [8]:
print("Naive Bayes Classifier performance on test data:")
f1_score, accuracy, recall, precision = evaluate_performance(test_labels, nb_predictions)
print(f"\tf1_score: {f1_score:.3f}, accuracy: {accuracy:.3f}")
print(f"\trecall: {recall:.3f}, precision: {precision:.3f}")
    
print("\nMNB Classifier performance on test data:")
for i in range(3):
    print(f"\tn = {i+1}")
    f1_score, accuracy, recall, precision = evaluate_performance(test_labels, mnb_predictions[i])
    print(f"\t\tf1_score: {f1_score:.3f}, accuracy: {accuracy:.3f}")
    print(f"\t\trecall: {recall:.3f}, precision: {precision:.3f}")

Naive Bayes Classifier performance on test data:
	f1_score: 0.698, accuracy: 0.860
	recall: 0.536, precision: 1.000

MNB Classifier performance on test data:
	n = 1
		f1_score: 0.965, accuracy: 0.979
		recall: 0.966, precision: 0.964
	n = 2
		f1_score: 0.975, accuracy: 0.985
		recall: 0.987, precision: 0.963
	n = 3
		f1_score: 0.945, accuracy: 0.965
		recall: 0.989, precision: 0.904
